In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import numpy as np

!python -m spacy download en_core_web_sm

2021-07-30 11:57:45.086684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-30 11:57:45.086705: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def normalization(embeds):
  norms = np.linalg.norm(embeds, 2, axis=1, keepdims=True)
  return embeds/norms


In [3]:
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")


2021-07-30 11:58:36.133746: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-30 11:58:36.169838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-30 11:58:36.170220: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0b:00.0 name: GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.845GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 462.00GiB/s
2021-07-30 11:58:36.170282: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-30 11:58:36.170332: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic librar

In [4]:
english_sentences = tf.constant(["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."])
italian_sentences = tf.constant(["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."])


In [41]:
def embedd_sents(sents):
    return normalization(encoder(preprocessor(tf.constant(sents)))["default"])

In [31]:
english_embeds = encoder(preprocessor(english_sentences))["default"]
italian_embeds = encoder(preprocessor(italian_sentences))["default"]


In [7]:
english_embeds = normalization(english_embeds)
italian_embeds = normalization(italian_embeds)


In [8]:
print (np.matmul(english_embeds, np.transpose(italian_embeds)))


[[0.6319289  0.30619788 0.4429748 ]
 [0.11652693 0.8596667  0.35940555]
 [0.14803988 0.32447952 0.9542651 ]]


In [9]:
import glob

In [13]:
texts = []
for p in glob.glob('../../data/ethics-ai-table/*.txt'):
    with open(p, 'r') as f:
        t = f.read()
        texts.append(t)

In [19]:
import spacy

texts

["\nOur Goals\n1\nDevelop and share best practices\n\nSupport research, discussions, identification, sharing, and recommendation of best practices in the research, development, testing, and fielding of AI technologies. Address such areas as fairness and inclusivity, explanation and transparency, security and privacy, values and ethics, collaboration between people and AI systems, interoperability of systems, and of the trustworthiness, reliability, containment, safety, and robustness of the technology.\n2\nAdvance public understanding\n\nAdvance wider public understanding and awareness of AI by sharing insights into AI’s core technologies, potential benefits – and costs. We will act as a trusted experts on AI for society and their leaders, and will work to increase public understanding of how AI is progressing.\n3\nProvide an open and inclusive platform for discussion & engagement\n\nCreate and support opportunities for AI researchers and key stakeholders, including people in technolog

In [23]:
en = spacy.load('en_core_web_sm')

In [27]:
sents = list(en(texts[0]).sents)

In [36]:
s = [str(s) for s in sents]

In [42]:
e_sents =embedd_sents(s)

In [38]:
targets = ['privacy protection',
'fairness',
'transparency, openness']

In [43]:
e_tergets = embedd_sents(targets)

array([31,  1, 19])

In [64]:
def print_best_sents(e_targets, e_sents, sents, targets):
    r = np.matmul(e_tergets, np.transpose(e_sents))
    for j, (i, target) in enumerate(zip(list(np.argmax(r, axis=1)), targets)):
        print(target)
        print("SCORE:", r[j][i])
        print("BEST HIT:")
        print(sents[i])
        print('-'*30)

In [65]:
print_best_sents(e_tergets, e_sents, sents, targets)

privacy protection
SCORE: 0.28070253
BEST HIT:

    5
    Social and Societal Influences of AI

    AI advances will touch people and society in numerous ways, including potential influences on privacy, democracy, criminal justice, and human rights.
------------------------------
fairness
SCORE: 0.25957346
BEST HIT:
Address such areas as fairness and inclusivity, explanation and transparency, security and privacy, values and ethics, collaboration between people and AI systems, interoperability of systems, and of the trustworthiness, reliability, containment, safety, and robustness of the technology.
------------------------------
transparency, openness
SCORE: 0.32075033
BEST HIT:


    Researchers, officials, and the public should be sensitive to these possibilities and we should seek to develop methods that detect and correct those errors and biases, not replicate them.
------------------------------
